In [ ]:
import os
import dotenv

In [ ]:
import zeep
import json
from tap_tradera.client import TraderaClient

In [ ]:
import csv

In [ ]:
with open('.secrets/item_ids.csv', 'r') as f:
    reader = csv.reader(f)
    item_ids = [row[0] for row in reader]

In [ ]:
len(item_ids)

In [ ]:
dotenv.load_dotenv(".env")

In [ ]:
client = TraderaClient(
    app_id=os.getenv("TAP_TRADERA_APP_ID"), app_key=os.getenv("TAP_TRADERA_APP_KEY")
)

In [ ]:
items = [
    client.get_item(item_id) for item_id in item_ids
]
items[0]

In [ ]:
for item in items:
    json.dumps(item)

In [ ]:
import datetime
from decimal import Decimal
from genson import SchemaBuilder
from genson.schema.strategies import TypedSchemaStrategy

class CustomDateTime(TypedSchemaStrategy):
    """
    strategy for date-time formatted strings
    """
    JS_TYPE = 'string'
    PYTHON_TYPE = (str, type(u''))

    # create a new instance variable
    def __init__(self, node_class):
        super().__init__(node_class)
        self.format = "date-time"

    @classmethod
    def match_object(self, obj):
        super().match_object(obj)
        try:
            if isinstance(obj, datetime.datetime):
                return True
            else:
                return False
        except (TypeError, ValueError) as exception:
            #print(exception)
            return False

    def to_schema(self):
        schema = super().to_schema()
        schema['type'] = self.JS_TYPE
        schema['format'] = self.format
        return schema

class CustomDecimal(TypedSchemaStrategy):
    """
    strategy for date-time formatted strings
    """
    JS_TYPE = 'number'
    PYTHON_TYPE = (Decimal, )

    @classmethod
    def match_object(self, obj):
        super().match_object(obj)
        try:
            if isinstance(obj, Decimal):
                return True
            else:
                return False
        except (TypeError, ValueError) as exception:
            #print(exception)
            return False

    def to_schema(self):
        schema = super().to_schema()
        schema['type'] = self.JS_TYPE
        return schema
    
class CustomSchemaBuilder(SchemaBuilder):
    """ detects & labels date-time formatted strings """
    EXTRA_STRATEGIES = (CustomDateTime, CustomDecimal, )

In [ ]:
builder = CustomSchemaBuilder()
for search_item in items:
    builder.add_object(search_item)
json_schema = builder.to_json(indent=2)
print(json_schema)

In [ ]:
with open("tap_tradera/schemas/item.json", "w") as item_schema:
    item_schema.write(json_schema)